In [98]:
import pandas as pd
import re
from itertools import chain

In [99]:
data_dir = '/home/admin-/Documents/dict_data/'
dt = '20190130'
user_dict_names = ['colour_user_dict_' + dt + '.txt', 'freq_user_dict_' + dt + '.txt']

In [100]:
def get_user_dicts(user_dict_names):
    sr_user_dicts = pd.Series()
    for user_dict in user_dict_names:
        file_path = data_dir + user_dict
        sr_user_dicts = sr_user_dicts.append(pd.read_csv(file_path, names=['line']))
    return sr_user_dicts

In [101]:
df_user_dicts = get_user_dicts(user_dict_names)

/home/admin-/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [102]:
df_user_dicts.line = df_user_dicts.line.apply(lambda x: x.lower())
df_user_dicts = df_user_dicts.drop_duplicates().sort_values(by=['line'])
df_user_dicts = df_user_dicts.reset_index(drop=True)
sr_user_dicts = df_user_dicts.line


In [103]:
def drop_duplicates_data(sr_user_dicts):
    drop_index_list = []

    for i in range(len(sr_user_dicts)-1):
        if i == len(sr_user_dicts)-1: 
            break
        prev = sr_user_dicts[i].split(" ")
        nex = sr_user_dicts[i+1].split(" ")
        if prev[0] == nex[0]:
            drop_index_list.append(i)
    
    sr_user_dicts = sr_user_dicts.drop(drop_index_list).reset_index(drop=True)
    return sr_user_dicts

In [104]:
sr_user_dicts = drop_duplicates_data(sr_user_dicts)

In [105]:
sr_user_dicts.to_csv(data_dir + 'user_dict_' + dt + '.txt', index=False)